In [ ]:
# Load libraries
import os,cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from keras import backend as K
K.set_image_dim_ordering('tf') # order channels by tensorflow

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam


In [ ]:
# cd Desktop
PATH = os.getcwd()
# Define data path
data_path = PATH + '/CNN'
data_dir_list = os.listdir(data_path) #Folders

# Define image size to feed into network
img_rows=128
img_cols=128
# Want grayscale image so only one channel (3 channels is rgb)
num_channel=1
num_epoch=20

# Define the number of classes (goats, elephants, dogs, cats)
num_classes = 4

img_data_list=[]

# Go through folders of pictures
for dataset in data_dir_list:
    # filter out hidden files
    if not dataset.startswith('.'):    
        img_list = os.listdir(data_path + '/' + dataset)
        print('Loaded the images of dataset-'+'{}\n'.format(dataset))
        # in each image in each folder: read it, convert to grayscale, resize to 128x128, and append it to list
        for img in img_list:
            # filter out hidden files
            if not img.startswith('.'):
                input_img = cv2.imread(data_path + '/' + dataset + '/' + img) # read
                input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY) # convert to grayscale
                input_img_resize = cv2.resize(input_img,(128, 128)) # resize
                img_data_list.append(input_img_resize) # append to list




In [ ]:
img_data = np.array(img_data_list) # convert into an array
img_data = img_data.astype('float32') # convert into a float
img_data /= 255 # Divide by 255 - normalization
print (img_data.shape)


# Reshape backend to feed into CNN. The number given to the axis value is where the
# number of channels stands in the order of samples, channels, rows, and columns (1 is first, 
# 4 is last). Different order for using tensorflow or theano.
if num_channel==1:
    if K.image_dim_ordering()=='th':
        img_data= np.expand_dims(img_data, axis=1) 
        print (img_data.shape)
    else:
        img_data= np.expand_dims(img_data, axis=4) 
        print (img_data.shape)

else:
    if K.image_dim_ordering()=='th':
        img_data=np.rollaxis(img_data,3,1)
        print (img_data.shape)
        

In [ ]:
# Assigning Labels

# Define the number of classes (4 for for our 4 classes of images)
num_classes = 4

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:102]=0
labels[102:206]=1
labels[204:306]=2
labels[306:]=3

names = ['cats','dogs','elephants','goats']
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

# Shuffle the dataset, random state gives a fixed value of shuffling
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset into x and y train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)


In [ ]:
# Building the convolutional neural network

input_shape=img_data[0].shape

model = Sequential()

# for first convolutional layer we use 32 filters, 3x3 rows by columns
# border_mode == 'same' is to preserve the size of the image even after convolution
model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=input_shape))
model.add(Activation('relu'))
# Another layer
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
# max pooling of size 2x2
model.add(MaxPooling2D(pool_size=(2, 2)))
# Dropout to avoid overfitting
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
# model.add(Convolution2D(64, 3, 3))
# model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# Flatten into an open one dimensional vector which becomes our feature vector to feed into the fully connected layer
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# number of classes is 4
model.add(Dense(num_classes))
# softmax regression
model.add(Activation('softmax'))


# compile by giving a loss function:
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=["accuracy"])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])

# Viewing model_configuration

model.summary()
model.get_config()
model.layers[0].get_config()
model.layers[0].input_shape
model.layers[0].output_shape
model.layers[0].get_weights()
np.shape(model.layers[0].get_weights()[0])
model.layers[0].trainable

In [ ]:
# Train the model
hist = model.fit(X_train, y_train, batch_size=16, nb_epoch=num_epoch, verbose=1, validation_data=(X_test, y_test))

In [ ]:
# evaluate the accuracy of the model

score = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

test_image = X_test[0:1]
print (test_image.shape)

print(model.predict(test_image))
print(model.predict_classes(test_image))
print(y_test[0:1])


In [ ]:
# Printing the confusion matrix
from sklearn.metrics import classification_report,confusion_matrix
import itertools

Y_pred = model.predict(X_test)
print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
# y_pred = model.predict_classes(X_test)
# print(y_pred)
target_names = ['class 0(cats)', 'class 1(Dogs)', 'class 2(elephants)','class 3(goats)']

print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))

print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))
